# Resources

> Managing daisyUI and Tailwind CSS resources for FastHTML projects

In [ ]:
#| default_exp core.resources

In [ ]:
#| export
from typing import List, Optional, Dict, Union
from enum import Enum
from dataclasses import dataclass, field
from fasthtml.common import Script, Link, Style, FT
from cjm_fasthtml_daisyui.core.types import CDNProvider

## Resource Management

The `DaisyUIResources` class provides flexible options for including daisyUI and Tailwind CSS in FastHTML projects. It supports:

1. **CDN delivery** - Quick setup for development and prototyping
2. **Local delivery** - Production-ready with compiled CSS
3. **Inline configuration** - Dynamic daisyUI configuration
4. **Version management** - Control specific versions of dependencies

In [ ]:
#| export
@dataclass
class ResourceVersions:
    """Version management for daisyUI and Tailwind CSS"""
    daisyui: str = "5"
    tailwind: str = "4"
    
    def get_daisyui_url(
        self,
        provider: CDNProvider = CDNProvider.JSDELIVR  # CDN provider to use for fetching resources
    ) -> str:  # Full CDN URL for daisyUI CSS
        """Get the full CDN URL for daisyUI"""
        base = provider.get_base_url()
        if provider == CDNProvider.JSDELIVR:
            return f"{base}/daisyui@{self.daisyui}"
        elif provider == CDNProvider.UNPKG:
            return f"{base}/daisyui@{self.daisyui}/dist/daisyui.css"
        else:
            # CDNJS doesn't host daisyUI, fallback to jsdelivr
            return CDNProvider.JSDELIVR.get_base_url() + f"/daisyui@{self.daisyui}"

    def get_daisyui_themes_url(
        self,
        provider: CDNProvider = CDNProvider.JSDELIVR  # CDN provider to use for fetching resources
    ) -> str:  # Full CDN URL for daisyUI themes CSS
        """Get the full CDN URL for daisyUI themes"""
        base = provider.get_base_url()
        if provider == CDNProvider.JSDELIVR:
            return f"{base}/daisyui@{self.daisyui}/themes.css"
        elif provider == CDNProvider.UNPKG:
            return f"{base}/daisyui@{self.daisyui}/dist/themes.css"
        else:
            # CDNJS doesn't host daisyUI, fallback to jsdelivr
            return CDNProvider.JSDELIVR.get_base_url() + f"/daisyui@{self.daisyui}/themes.css"
    
    def get_tailwind_url(
        self,
        provider: CDNProvider = CDNProvider.JSDELIVR  # CDN provider to use for fetching resources
    ) -> str:  # Full CDN URL for Tailwind CSS browser JavaScript
        """Get the full CDN URL for Tailwind CSS browser version"""
        base = provider.get_base_url()
        if provider == CDNProvider.JSDELIVR:
            return f"{base}/@tailwindcss/browser@{self.tailwind}"
        elif provider == CDNProvider.UNPKG:
            return f"{base}/@tailwindcss/browser@{self.tailwind}/browser.js"
        else:
            # CDNJS doesn't host Tailwind v4 browser version, fallback to jsdelivr
            return CDNProvider.JSDELIVR.get_base_url() + f"/@tailwindcss/browser@{self.tailwind}"

In [ ]:
#| export
class DaisyUIResources:
    """Manages daisyUI and Tailwind CSS resources for FastHTML projects"""
    
    @staticmethod
    def cdn_headers(
        versions: Optional[ResourceVersions] = None,  # Version configuration for daisyUI and Tailwind
        provider: CDNProvider = CDNProvider.JSDELIVR,  # CDN provider to use for fetching resources
        include_tailwind: bool = True,  # Whether to include Tailwind CSS browser JavaScript
        additional_css: Optional[List[str]] = None,  # List of additional CSS URLs to include
        additional_js: Optional[List[str]] = None  # List of additional JavaScript URLs to include
    ) -> List[FT]:  # List of FastHTML elements (Link and Script tags)
        """CDN-based resources for quick testing and development"""
        versions = versions or ResourceVersions()
        headers = []
        
        # Add daisyUI CSS
        headers.append(
            Link(rel="stylesheet", href=versions.get_daisyui_url(provider))
        )
        headers.append(
            Link(rel="stylesheet", href=versions.get_daisyui_themes_url(provider))
        )
        
        # Add Tailwind CSS browser version
        if include_tailwind:
            headers.append(
                Script(src=versions.get_tailwind_url(provider))
            )
        
        # Add any additional CSS files
        if additional_css:
            for css_url in additional_css:
                headers.append(Link(rel="stylesheet", href=css_url))
        
        # Add any additional JS files
        if additional_js:
            for js_url in additional_js:
                headers.append(Script(src=js_url))
        
        return headers

    @staticmethod
    def local_headers(
        css_path: str = "/static/styles.css",  # Path to compiled CSS file
        js_paths: Optional[List[str]] = None,  # List of paths to JavaScript files
        additional_css: Optional[List[str]] = None,  # List of additional CSS paths
        additional_js: Optional[List[str]] = None  # List of additional JavaScript paths
    ) -> List[FT]:  # List of FastHTML elements (Link and Script tags)
        """Local file-based resources for production use"""
        headers = []
        
        # Add main compiled CSS
        headers.append(Link(rel="stylesheet", href=css_path))
        
        # Add any JavaScript files
        if js_paths:
            for js_path in js_paths:
                headers.append(Script(src=js_path))
        
        # Add any additional CSS files
        if additional_css:
            for css_url in additional_css:
                headers.append(Link(rel="stylesheet", href=css_url))
        
        # Add any additional JS files
        if additional_js:
            for js_url in additional_js:
                headers.append(Script(src=js_url))
        
        return headers

    @staticmethod
    def inline_css(
        content: str,  # CSS content to embed inline
        id: Optional[str] = None  # Optional ID attribute for the style element
    ) -> Style:  # FastHTML Style element
        """Create an inline CSS style element"""
        attrs = {"id": id} if id else {}
        return Style(content, **attrs)

    @staticmethod
    def minimal_css(
    ) -> str:  # Minimal CSS string with Tailwind and daisyUI imports
        """Get minimal CSS for Tailwind v4 with daisyUI plugin"""
        return '''@import "tailwindcss";
@plugin "daisyui";'''

## Resource Presets

Common resource configurations for different use cases:

In [ ]:
#| export
class ResourcePresets:
    """Common resource configurations"""
    
    @staticmethod
    def development(
    ) -> List[FT]:  # List of FastHTML header elements for development
        """Quick development setup with CDN resources"""
        return DaisyUIResources.cdn_headers()
    
    @staticmethod
    def production(
        css_path: str = "/static/styles.css"  # Path to compiled production CSS file
    ) -> List[FT]:  # List of FastHTML header elements for production
        """Production setup with compiled CSS"""
        return DaisyUIResources.local_headers(css_path=css_path)
    
    @staticmethod
    def testing(
    ) -> List[FT]:  # List of FastHTML header elements for testing
        """Testing setup with fast CDN and no caching"""
        headers = DaisyUIResources.cdn_headers()
        # Add cache-busting meta tag
        headers.append(
            Link(rel="preload", href="https://cdn.jsdelivr.net/npm/daisyui@5", as_="style")
        )
        return headers
    
    @staticmethod
    def offline(
        css_path: str = "/static/daisyui.css",  # Path to local daisyUI CSS file
        tailwind_path: str = "/static/tailwind.js"  # Path to local Tailwind browser JavaScript
    ) -> List[FT]:  # List of FastHTML header elements for offline use
        """Completely offline setup with local files"""
        return [
            Link(rel="stylesheet", href=css_path),
            Script(src=tailwind_path)
        ]

## Usage Examples

Here are common usage patterns for including daisyUI resources in FastHTML projects:

### Basic Development Setup

In [ ]:
from fasthtml.common import fast_app

# Quick development setup
app, rt = fast_app(
    pico=False,  # Disable Pico CSS since we're using daisyUI
    hdrs=ResourcePresets.development()
)

# Or with explicit CDN configuration
app, rt = fast_app(
    pico=False,
    hdrs=DaisyUIResources.cdn_headers()
)

### Custom Version Control

In [ ]:
# Use specific versions
versions = ResourceVersions(daisyui="5.0.0", tailwind="4.0.0-alpha.25")

app, rt = fast_app(
    pico=False,
    hdrs=DaisyUIResources.cdn_headers(versions=versions)
)

# Or use a different CDN provider
app, rt = fast_app(
    pico=False,
    hdrs=DaisyUIResources.cdn_headers(
        provider=CDNProvider.UNPKG,
        versions=versions
    )
)

### Production Setup

In [ ]:
# Production with compiled CSS
app, rt = fast_app(
    pico=False,
    hdrs=ResourcePresets.production(css_path="/static/compiled.css")
)

# Production with additional resources
app, rt = fast_app(
    pico=False,
    hdrs=DaisyUIResources.local_headers(
        css_path="/static/main.css",
        js_paths=["/static/app.js"],
        additional_css=["/static/animations.css"]
    )
)

### Hybrid Approach with Inline Configuration

In [ ]:
# This will be implemented in config.ipynb
# from cjm_fasthtml_daisyui.core.config import DaisyUIConfig

# For now, show the pattern:
custom_css = """
@import "tailwindcss";
@plugin "daisyui" {
  themes: cupcake --default, dark --prefersdark;
}
"""

app, rt = fast_app(
    pico=False,
    hdrs=[
        DaisyUIResources.inline_css(custom_css),
        *DaisyUIResources.cdn_headers(include_tailwind=False)
    ]
)

## Advanced Features

In [ ]:
#| export
@dataclass
class ResourceOptimization:
    """Advanced resource optimization options"""
    preload: bool = True
    prefetch: bool = False
    async_load: bool = False
    defer: bool = False
    integrity: Optional[str] = None
    crossorigin: Optional[str] = "anonymous"
    
    def apply_to_link(
        self,
        link_attrs: Dict[str, str]  # Dictionary of link element attributes
    ) -> Dict[str, str]:  # Updated dictionary with optimization attributes applied
        """Apply optimization attributes to a link element"""
        if self.preload:
            link_attrs["rel"] = "preload"
            link_attrs["as"] = "style"
        elif self.prefetch:
            link_attrs["rel"] = "prefetch"
        
        if self.integrity:
            link_attrs["integrity"] = self.integrity
        
        if self.crossorigin:
            link_attrs["crossorigin"] = self.crossorigin
            
        return link_attrs
    
    def apply_to_script(
        self,
        script_attrs: Dict[str, str]  # Dictionary of script element attributes
    ) -> Dict[str, str]:  # Updated dictionary with optimization attributes applied
        """Apply optimization attributes to a script element"""
        if self.async_load:
            script_attrs["async"] = "true"
        elif self.defer:
            script_attrs["defer"] = "true"
        
        if self.integrity:
            script_attrs["integrity"] = self.integrity
        
        if self.crossorigin:
            script_attrs["crossorigin"] = self.crossorigin
            
        return script_attrs

In [ ]:
#| export
class ResourceManager:
    """
    Advanced resource manager with caching and optimization
    """
    def __init__(self):
        """Initialize resource manager with cache and default optimization settings"""
        self._cache: Dict[str, List[FT]] = {}
        self._optimization = ResourceOptimization()
    
    def get_optimized_headers(
        self,
        key: str = "default",  # Cache key to use for storing/retrieving headers
        force_refresh: bool = False  # Whether to bypass cache and rebuild headers
    ) -> List[FT]:  # List of cached or newly built FastHTML header elements
        """Get cached headers with optimization"""
        if key not in self._cache or force_refresh:
            self._cache[key] = self._build_optimized_headers()
        return self._cache[key]
    
    def _build_optimized_headers(
        self
    ) -> List[FT]:  # List of optimized FastHTML header elements
        """Build optimized headers"""
        # This would be implemented based on specific needs
        return DaisyUIResources.cdn_headers()
    
    def clear_cache(
        self
    ) -> None:
        """Clear the resource cache"""
        self._cache.clear()

## Best Practices

1. **Development**: Use CDN resources for quick iteration
2. **Production**: Compile CSS with only used utilities for optimal performance
3. **Testing**: Use consistent versions to ensure reproducibility
4. **Offline**: Download and serve files locally for environments without internet

## Testing the Resources

In [ ]:
# Test CDN header generation
cdn_headers = DaisyUIResources.cdn_headers()
print(f"Generated {len(cdn_headers)} headers:")
for header in cdn_headers:
    print(f"  - {header}")

# Test with specific versions
custom_headers = DaisyUIResources.cdn_headers(
    versions=ResourceVersions(daisyui="5.1.0", tailwind="4.0.0-beta.1")
)
print(f"\nCustom version headers: {len(custom_headers)}")

Generated 3 headers:
  - <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5">
  - <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/themes.css">
  - <script src="https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4"></script>

Custom version headers: 3


In [ ]:
# Test resource presets
dev_headers = ResourcePresets.development()
prod_headers = ResourcePresets.production()
test_headers = ResourcePresets.testing()

print(f"Development: {len(dev_headers)} headers")
print(f"Production: {len(prod_headers)} headers")
print(f"Testing: {len(test_headers)} headers")

Development: 3 headers
Production: 1 headers
Testing: 4 headers


In [ ]:
# Test minimal CSS generation
minimal = DaisyUIResources.minimal_css()
print("Minimal CSS:")
print(minimal)

Minimal CSS:
@import "tailwindcss";
@plugin "daisyui";


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()